In [1]:
import premise as ps

In [2]:
import bw2io as bi
import bw2data as bd

In [3]:
bd.projects.set_current('MCC')

In [4]:
bi.bw2setup()

Biosphere database already present!!! No setup is needed


In [5]:
ei391cdir = "/home/haithamth/Documents/ecoinvent/ecoin_cuttoff_391/ecoinvent 3.9.1_cutoff_ecoSpold02/datasets"
data_base_name = "ecoinvent-3.9.1-cuttoff"
if data_base_name in bd.databases:
    print("Database has already been imported")
else:
    ei391c = bi.SingleOutputEcospold2Importer(ei391cdir, data_base_name)
    ei391c.apply_strategies()
    ei391c.statistics()

    #ei39.drop_unlinked(True)
    ei391c.write_database()

Database has already been imported


In [6]:
from premise import NewDatabase

In [7]:
# ndb = NewDatabase(
#     scenarios=[
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2020},
#         {"model":"image", "pathway":"SSP2-RCP26", "year":2020},
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2030},
#         {"model":"image", "pathway":"SSP2-RCP26", "year":2030},
#         {"model":"image", "pathway":"SSP2-RCP19", "year":2040},
#         {"model":"image", "pathway":"SSP2-RCP26", "year":2040},
#     ],
#     source_db="ecoinvent-3.9.1-cuttoff", # <-- name of the database in the BW2 project. Must be a string.
#     source_version="3.9.1", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
#     key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=', # <-- decryption key
#     # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
#     use_multiprocessing=True # <-- set to True if you want to use multiprocessing
# )

In [8]:
# ndb.update_electricity()

In [9]:
# ndb.write_db_to_brightway(["SSP2-RCP19_2020","SSP2-RCP26_2020", "SSP2-RCP19_2030","SSP2-RCP26_2030", "SSP2-RCP19_2040","SSP2-RCP26_2040"])

In [10]:
ecn = bd.Database("ecoinvent-3.9.1-cuttoff")

In [11]:
ecn.search('steel unalloyed')[0].key

('ecoinvent-3.9.1-cuttoff', 'd872e0d78319cb13e12b96de83e19dd7')

In [12]:
import uuid
class Product:
    '''This class represents a product in the system, it should take care of the following:
    - Create a unique id for each product
    - Create a unique serial number for each product
    - Keep track of the product's service life
    '''
    instances = []
    def __init__(self, name: str, amount: float | int, unit: str, service_life: int, production_lci: tuple[str,str]):
        # randomly generated id
        self.id = uuid.uuid4().hex
        self.name = name
        self.amount = amount
        self.unit = unit
        self.production_lci = production_lci
        self.service_life = service_life
        self.instances.append(self)

    def __repr__(self):
        return f"{(self.name, self.id)}"
    
    def replacement_sn(self):
        ...

In [13]:
p = Product(name='steel_beam', amount=1, unit='kg', service_life=20, production_lci=('ecoinvent-3.9.1-cuttoff', 'd872e0d78319cb13e12b96de83e19dd7'))

In [14]:
p2 = Product(name='steel_beam2', amount=2, unit='kg', service_life=5, production_lci=('ecoinvent-3.9.1-cuttoff', 'd872e0d78319cb13e12b96de83e19dd7'))

In [15]:
class Mfa:
    '''
    This class represents a material flow analysis, it should take care of the following:
    - Create a time line for the analysis
    - Create a list of points in time where replacements occur
    '''
    def __init__(self, start: int = 2020, end: int = 2080, time_step: int = 10):
        self.start = start
        self.end = end
        self.time_step = time_step
        self.time_line = list(range(self.start, self.end+1, self.time_step))
        self.points = []
        self.impacts = {}
    
    def add_points(self, product: Product):
        first_repl = product.service_life + self.start
        
        for i in range(first_repl, self.end+1, product.service_life):
            self.points.append((i, product))
        # clean duplicates
        self.points = list(set(self.points))
        # sort by year
        self.points = sorted(self.points, key=lambda x: x[0])
        return self.points

In [16]:
tl = Mfa(start=2020, end=2050)

In [17]:
tl.add_points(p)

[(2040, ('steel_beam', '20579dc1926b492ab9c110434b00deda'))]

In [18]:
tl.add_points(p2)

[(2025, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2030, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2035, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2040, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2040, ('steel_beam', '20579dc1926b492ab9c110434b00deda')),
 (2045, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2050, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8'))]

In [19]:
tl.points

[(2025, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2030, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2035, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2040, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2040, ('steel_beam', '20579dc1926b492ab9c110434b00deda')),
 (2045, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2050, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8'))]

In [75]:
import bw2calc as bc
class ProLCA:
    '''
    This class should take care of the following:
    - Create a production lca for each product
    - Create a time line based lca for each product
    - Create a future lca for each product
    - Create a future lca for the whole system
    '''
    def __init__(self, products: list[Product], mfa: Mfa, methods: list[tuple[str,str,str]] = [('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)')]):
        self.products = products
        self.mfa = mfa
        self.methods = methods
        self.yearly_databases = {}
    
    def production_lca(self, db: bd.Database = bd.Database("ecoinvent-3.9.1-cuttoff")):
        results_dict = {}
        for product in self.products:
            impacts = []
            temp_product_dict = {str(product): []}
            results_dict.update(temp_product_dict)
            # get the activity from the database
            # create a functional unit
            activity = db.get(product.production_lci)
            amount = product.amount
            fu = {activity: amount}
            lca = bc.LCA(fu, self.methods[0])
            lca.lci()
            lca.lcia()
            impacts.append(lca.score)
            if len(self.methods) > 1:
                for method in self.methods[1:]:
                    lca.switch_method(method)
                    lca.lcia()
                    impacts.append(lca.score)
                    results_dict[str(product)] = impacts
            else:
                results_dict[str(product)] = impacts
        return results_dict
    
    def repl_lca(self):
        # replacement lcas
        # get the timeline
        time_line = self.time_line.points
    
    def database_chooser(self, year):
        available_years = self.yearly_databases.keys()
        # make sure that the function that create_time_line_dict is called before this function and a dictionary of yearly databases is created
        assert len(available_years) > 0, "Please create a dictionary of yearly databases first"
        
        if year in available_years:
            return self.yearly_databases[year]
        elif year < min(available_years):
            return self.yearly_databases[min(available_years)]
        elif year > max(available_years):
            return self.yearly_databases[max(available_years)]
        else:
            # round down to the nearest year
            return self.yearly_databases[year - (year % 10)]
        
            # for y in available_years:
            #     if y > year:
            #         return self.yearly_databases[y]
          
    
    # def time_line_based_lca(self):
    #     results_dict = {}
    #     for point in self.mfa.points:
    #         for product in self.products:
    #             impacts = []
    #             if point[1] == product:
    #                 dbs = self.database_chooser(year=point[0])
    #                 for db in dbs:
    #                     fetched_db = bd.Database(str(db))
    #                     results_dict[str(point[0]) + "_" + str(product) + "_" + str(db)] = []
                        

    #                     activity = fetched_db.get(product.production_lci[1])

    #                     amount = product.amount
    #                     fu = {activity: amount}
                        
    #                     lca = bc.LCA(fu, self.methods[0])
                        
    #                     lca.lci()
    #                     lca.lcia()
                        
    #                     if len(self.methods) > 1:
                            
    #                         for method in self.methods[1:]:
    #                             lca.switch_method(method)
    #                             lca.lcia()
    #                             impacts.append(lca.score)
    #                             results_dict[str(point[0]) + "_" + str(product) + "_" + str(db)] = impacts
                                
    #                     else:
    #                         impacts = [lca.score]
    #                         results_dict[str(point[0]) + "_" + str(product) + "_" + str(db)] = impacts
                        
    #     return results_dict
    
    
    def time_line_based_lca(self):
        results_dict = {}
        for point in self.mfa.points:
            for product in self.products:
                # Check if the second element in `point` equals `product`
                if point[1] == product:
                    # Choose the appropriate database(s) based on the year
                    dbs = self.database_chooser(year=point[0])
                    
                    # Iterate through all chosen databases
                    for db in dbs:
                        # Construct a unique key for storing results
                        result_key = f"{point[0]}_{product}_{db}"
                        # Ensure fetched_db returns the expected database
                        fetched_db = bd.Database(str(db))
                        
                        # Ensure this returns the expected activity
                        activity = fetched_db.get(product.production_lci[1])
                        
                        # Define the functional unit for the LCA
                        amount = product.amount
                        fu = {activity: amount}
                        
                        # Initialize the LCA with the first method and calculate impacts
                        lca = bc.LCA(fu, self.methods[0])
                        lca.lci()
                        lca.lcia()
                        
                        # Initialize a list to store impact scores for all methods
                        impacts = [lca.score]
                        
                        # If there are additional methods, switch methods and calculate additional impacts
                        if len(self.methods) > 1:    
                            for method in self.methods[1:]:
                                lca.switch_method(method)
                                lca.lcia()
                                impacts.append(lca.score)
                        
                        # Store all calculated impacts in the results dictionary
                        results_dict[result_key] = impacts
        
        return results_dict

                    
    def create_time_line_dict(self):
        yearly_databases = {}
        for year in tl.time_line:
            yearly_databases[year] = []
        for db in bd.databases:
            db_processed = str(db).split("_")
            if db_processed[0] == 'ecoinvent-3.9.1-cuttoff':
                ecoinvent = db
            elif len(db_processed) > 1:
                yearly_databases[int(db_processed[1])].append(db)
        self.yearly_databases = yearly_databases
        

In [76]:
lca = ProLCA(products=[p, p2], mfa=tl, methods=[('EF v3.0 EN15804', 'acidification', 'accumulated exceedance (AE)'),
 ('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)')])

In [ ]:
#jjj

In [77]:
lca.create_time_line_dict()

In [78]:
lca.time_line_based_lca()

{"2025_('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')_SSP2-RCP19_2020": [0.013595427150702041,
  3.754028157400075],
 "2025_('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')_SSP2-RCP26_2020": [0.01359542716367803,
  3.7540281581736594],
 "2030_('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')_SSP2-RCP19_2030": [0.012321920894703289,
  3.409380364928732],
 "2030_('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')_SSP2-RCP26_2030": [0.013263918767324615,
  3.6783845363499763],
 "2035_('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')_SSP2-RCP19_2030": [0.012321920894703289,
  3.409380364928732],
 "2035_('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')_SSP2-RCP26_2030": [0.013263918767324615,
  3.6783845363499763],
 "2040_('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')_SSP2-RCP19_2040": [0.011980899605036212,
  3.328349634396882],
 "2040_('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')_SSP2-RCP26_2040": [0.012513544960288957,
  3.5479421845669474],
 "2040_('steel_beam',

In [53]:
tl.points

[(2025, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2030, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2035, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2040, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2040, ('steel_beam', '20579dc1926b492ab9c110434b00deda')),
 (2045, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8')),
 (2050, ('steel_beam2', '398ce967a2a54e0fa75ce41ba91800f8'))]

In [ ]:
for p in tl.points:
    print(p[1])

('steel_beam', '126a254a6e534689b127bbdef01d14b5')
('steel_beam2', '1b399e7abe7e440fb86f1e6169c9cbed')
('steel_beam', '126a254a6e534689b127bbdef01d14b5')
('steel_beam', '126a254a6e534689b127bbdef01d14b5')


In [ ]:
lca = ProLCA(products=[p, p2], mfa=tl, methods=[('EF v3.0 EN15804', 'acidification', 'accumulated exceedance (AE)'),
 ('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)')])

In [ ]:
lca.database_chooser(2020)

TypeError: ProLCA.database_chooser() takes 1 positional argument but 2 were given

In [ ]:
ProLCA(products=[p, p2], mfa=tl, methods=[('EF v3.0 EN15804', 'acidification', 'accumulated exceedance (AE)'),
 ('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)')]).production_lca()

{"('steel_beam', '126a254a6e534689b127bbdef01d14b5')": [0.007053131262017521,
  1.8778845564475808],
 "('steel_beam2', '1b399e7abe7e440fb86f1e6169c9cbed')": [0.014106262524035042,
  3.7557691128951616]}

In [ ]:
ProLCA(products=[p, p2], mfa=tl, methods=[('EF v3.0 EN15804', 'acidification', 'accumulated exceedance (AE)'),
 ('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)')]).time_line_based_lca()

2040 <class 'tuple'>
2053 <class 'tuple'>
2060 <class 'tuple'>
2080 <class 'tuple'>


In [ ]:
lca = ProLCA(product=p, time_line=tl, methods=[('EF v3.0 EN15804', 'acidification', 'accumulated exceedance (AE)'),
 ('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)')])

In [ ]:
lca.time_line.points

[(2040, ('steel_beam', '126a254a6e534689b127bbdef01d14b5')),
 (2053, ('steel_beam2', '1b399e7abe7e440fb86f1e6169c9cbed')),
 (2060, ('steel_beam', '126a254a6e534689b127bbdef01d14b5')),
 (2080, ('steel_beam', '126a254a6e534689b127bbdef01d14b5'))]

In [ ]:
def db_chooser(year: int):
    # choose the database based on the year
    if year < 2030:
        return "SSP2-RCP19_2020"
    elif year < 2040:
        return "SSP2-RCP19_2030"

In [ ]:
import premise

In [ ]:
def activity_mapper(activity):
    ...

In [ ]:
db1 = bd.Database("SSP2-RCP19_2020")
db2 = bd.Database("SSP2-RCP19_2030")
db_eco = bd.Database("ecoinvent-3.9.1-cuttoff")

In [ ]:
stl1= db_eco.get("d872e0d78319cb13e12b96de83e19dd7")

In [ ]:
stl2 = db2.get("d872e0d78319cb13e12b96de83e19dd7")

In [ ]:
# run lca on stl1 and stl2
methd = ('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)')

stl1_lca = bc.LCA({stl1: 1}, methd)

stl1_lca.lci()

stl1_lca.lcia()

stl1_lca.score

stl2_lca = bc.LCA({stl2: 1}, methd)

stl2_lca.lci()

stl2_lca.lcia()

stl2_lca.score

stl1_lca.score - stl2_lca.score


0.1731943739832147

In [ ]:
[a for a in bd.methods if 'EN' in a[0]]

[('EF v3.0 EN15804', 'acidification', 'accumulated exceedance (AE)'),
 ('EF v3.0 EN15804', 'climate change', 'global warming potential (GWP100)'),
 ('EF v3.0 EN15804',
  'climate change: biogenic',
  'global warming potential (GWP100)'),
 ('EF v3.0 EN15804',
  'climate change: fossil',
  'global warming potential (GWP100)'),
 ('EF v3.0 EN15804',
  'climate change: land use and land use change',
  'global warming potential (GWP100)'),
 ('EF v3.0 EN15804',
  'ecotoxicity: freshwater',
  'comparative toxic unit for ecosystems (CTUe)'),
 ('EF v3.0 EN15804',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('EF v3.0 EN15804',
  'eutrophication: freshwater',
  'fraction of nutrients reaching freshwater end compartment (P)'),
 ('EF v3.0 EN15804',
  'eutrophication: marine',
  'fraction of nutrients reaching marine end compartment (N)'),
 ('EF v3.0 EN15804',
  'eutrophication: terrestrial',
  'accumulated exceedance (AE)'),
 ('EF v3.0 EN15804',
  'hum